In [1]:
import csv
import random
from tkinter import *

class DinnerMenuRecommendationSystem:
    def __init__(self, root):
        self.root = root
        self.root.title("Dinner Menu Recommendation System")
        self.root.geometry("800x600")

        self.theme_label = Label(root, text="먹고 싶은 음식의 테마를 선택하세요(한식,일식,중식):")
        self.theme_label.pack()

        self.theme_var = StringVar()
        self.theme_entry = Entry(root, textvariable=self.theme_var)
        self.theme_entry.pack()

        self.type_label = Label(root, text="먹고 싶은 종류를 선택하세요:\n(밥, 고기, 찌개, 떡, 전, 면, 회, 술, 채소, 국, 튀김, 두부, 만두, 해산물)")
        self.type_label.pack()

        self.type_var = StringVar()
        self.type_entry = Entry(root, textvariable=self.type_var)
        self.type_entry.pack()

        self.recommend_button = Button(root, text="음식 재료 추천받기", command=self.recommend_dinner)
        self.recommend_button.pack()

        self.ingredients_label = Label(root, text="")
        self.ingredients_label.pack()

        self.user_ingredients_label = Label(root, text="맘에 드는 재료를 선택하세요 (쉼표로 구분): ")
        self.user_ingredients_label.pack()

        self.user_ingredients_var = StringVar()
        self.user_ingredients_entry = Entry(root, textvariable=self.user_ingredients_var)
        self.user_ingredients_entry.pack()

        self.user_recommend_button = Button(root, text="음식 추천 받기", command=self.recommend_by_user_input)
        self.user_recommend_button.pack()

        self.result_label = Label(root, text="")
        self.result_label.pack()

        self.header, self.csv_data = self.read_csv('Food_recipe.csv')

    def read_csv(self, file_path):
        data = []
        with open(file_path, newline='', encoding='utf-8') as csvfile:
            reader = csv.reader(csvfile)
            header = next(reader)
            for row in reader:
                data.append(row)
        return header, data

    def filter_food(self, theme, food_type):
        filtered_data = []
        for row in self.csv_data:
            if row[1] == theme and row[2] == food_type:
                filtered_data.append(row)
        return filtered_data

    def recommend_ingredients(self, filtered_data):
        all_ingredients = [ingredient for food in filtered_data for ingredient in food[3:] if ingredient]
        
        # all_ingredients가 비어 있는 경우를 처리
        if not all_ingredients:
            return []

        # min(3, len(all_ingredients))를 사용하여 샘플 크기를 설정
        recommended_ingredients = random.sample(all_ingredients, min(3, len(all_ingredients)))
        return recommended_ingredients

    def recommend_dinner(self):
        theme_choice = self.theme_var.get()
        type_choice = self.type_var.get()

        filtered_data = self.filter_food(theme_choice, type_choice)

        if not filtered_data:
            self.ingredients_label.config(text="해당하는 음식이 없습니다.")
            return

        recommended_ingredients = self.recommend_ingredients(filtered_data)
        self.ingredients_label.config(text=f"추천 음식 재료: {', '.join(recommended_ingredients)}")

    def recommend_dishes_helper(self, selected_ingredients):
        theme_choice = self.theme_var.get()
        type_choice = self.type_var.get()

        recommended_dishes = [
            row[0] for row in self.csv_data
            if row[1] == theme_choice
            and row[2] == type_choice
            and all(ingredient in row[3:] for ingredient in selected_ingredients)
        ]

        return recommended_dishes

    def recommend_by_user_input(self):
        selected_ingredients = [ingredient.strip() for ingredient in self.user_ingredients_var.get().split(',')]

        if not selected_ingredients:
            self.result_label.config(text="선택된 재료가 없습니다. 다시 입력해주세요.")
            return

        try:
            recommended_dishes = self.recommend_dishes_helper(selected_ingredients)

            if not recommended_dishes:
                self.result_label.config(text="해당하는 음식이 없습니다. 다시 입력해주세요.")
                return

            num_recommendations = min(3, len(recommended_dishes))
            if num_recommendations > 0:
                result_str = "\n추천 음식:\n"
                for i, food in enumerate(random.sample(recommended_dishes, num_recommendations), start=1):
                    result_str += f"{i}. {food}\n"
                self.result_label.config(text=result_str)
            else:
                self.result_label.config(text="해당하는 음식이 없습니다. 다시 입력해주세요.")

        except Exception as e:
            print(f"Error in recommend_by_user_input: {e}")
            self.result_label.config(text="오류가 발생했습니다. 다시 시도해주세요.")

if __name__ == "__main__":
    root = Tk()
    app = DinnerMenuRecommendationSystem(root)
    root.mainloop()
